In [4]:
cd /content/drive/MyDrive/commit_folder/chungwadae/torch_nlp

/content/drive/MyDrive/commit_folder/chungwadae/torch_nlp


In [1]:
# import basic libarary
import pandas
import pickle
import math

# import torch module
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self, dimension, vocab_size, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(vocab_size, dimension)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, dimension, 2).float()*(-math.log(10000.0)/dimension)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class TransoformerClassifierNetwork(nn.Module):
    def __init__(self, nhead, dim_feedforward, num_layers, dropout, vocab_size, dimension):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, dimension)

        self.pos_encoder = PositionalEncoding(
            dimension = dimension,
            dropout = dropout,
            vocab_size = vocab_size
        )

        encoder_layer = TransformerEncoderLayer(
            d_model = dimension,
            nhead = nhead,
            dim_feedforward = dim_feedforward,
            dropout = dropout
        )

        self.transformer_encoder = TransformerEncoder(
            encoder_layer,
            num_layers = num_layers
        )

        self.linear = nn.Linear(dimension, 3)
        self.dimension = dimension

    def forward(self, input):
        x = self.emb(input)*math.sqrt(self.dimension)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        output = self.linear(x)
        return output


In [13]:
train_data_path = 'data/train_data.pkl'
train_label_path = 'data/train_label.pkl'
test_data_path = 'data/test_data.pkl'

with open(train_data_path, 'rb') as f:
  train_data = pickle.load(f)

with open(train_label_path, 'rb') as f:
  train_label = pickle.load(f)

with open(test_data_path, 'rb') as f:
  test_data = pickle.load(f)

In [18]:
EPOCHS = 50
model = TransoformerClassifierNetwork(
    nhead = 8,
    dim_feedforward = 50,
    num_layers = 6,
    dropout = 0.2,
    vocab_size = 42000,
    dimension = 512
)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)